In [ ]:
import numpy as np
import pandas as pd
from numba import njit
import matplotlib.pyplot as plt
import os
import glob
import shutil
import json
import shlex
import subprocess
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10,8)

In [ ]:
input_file_path="/groups/sxs/hchaudha/scripts/observe_from_checkpoint_files/Gaugecheck_small_bbh.input"

save_folder="/groups/sxs/hchaudha/spec_runs/observer_from_checkpoints/"
base_path="/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/"
Lev=1

In [ ]:
def get_parent(full_path,parent_level):
  arr = full_path.split("/")[:-parent_level]
  path=""
  for i in arr:
    path += i + "/"
    
  return path

In [ ]:
checkpoint_pattern = base_path+f"Ev/Lev{Lev}_??/Run/Checkpoints/*"
checkpoint_path_collection = []
saved_checkpoint_path=[]
for folder_name in glob.iglob(checkpoint_pattern, recursive=True):
    if os.path.isdir(folder_name):
        checkpoint_path_collection.append(folder_name)
        saved_checkpoint_path.append(save_folder+folder_name.split("/")[-1])
        # Copy the checkpoint folder
        shutil.copytree(folder_name,save_folder+folder_name.split("/")[-1])
        print(f"Copied: {folder_name}")


        files_path_collection = []
        files_pattern = get_parent(folder_name,2)+"*.input"
        for file_path in glob.iglob(files_pattern, recursive=True):
            if os.path.isfile(file_path):
                files_path_collection.append(file_path)

        files_pattern = get_parent(folder_name,2)+"*.txt"
        for file_path in glob.iglob(files_pattern, recursive=True):
            if os.path.isfile(file_path):
                files_path_collection.append(file_path)


        for file in files_path_collection:
            shutil.copy(file,save_folder+folder_name.split("/")[-1])

        # ApplyObserver path
        applyObserverPath = base_path+"bin/ApplyObservers"
        # Copy ApplyObserver
        shutil.copy(applyObserverPath,save_folder+folder_name.split("/")[-1])


with open(save_folder+"paths.json",'w') as f:
    json.dump(checkpoint_path_collection,f,indent=2)

In [ ]:
def ApplyObserver(checkpoint_path,input_file_path):
  command=f"""cd {checkpoint_path} && pwd && ./ApplyObservers -t psi,kappa,H,InitGridHi,InitHhatt,Theta -r 11,122,1,1,Scalar,1 -d 4,4,4,3,1,4 -domaininput "./GrDomain.input" -h5prefix "Cp-VarsGr" {input_file_path}"""
  return command

In [ ]:
# def split(a, n):
#     k, m = divmod(len(a), n)
#     return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

# list(split(saved_checkpoint_path,4))[0]


# Run apply observers in each folder
for folder_path in saved_checkpoint_path:
  output=subprocess.run(ApplyObserver(folder_path,input_file_path),capture_output=True, shell=True,text=True)
  print(output.stderr)
  print(f"Completed folder {folder_path}\n")

## Combine pvd files

In [ ]:
def read_pvd_file(file_path):
  with open(file_path) as f:
    data = f.readlines()
    write_full_vtk_file_path(data,file_path)
    return data


def write_full_vtk_file_path(read_data,file_path):
  vtk_folder_name= file_path.split("/")[-1][:-4]
  vtk_folder_path= file_path[:-4]
  for i,path in enumerate(read_data):
    read_data[i] = path.replace(vtk_folder_name,vtk_folder_path)


def add_lists_skip_random(a,b):
  for i in b:
    if "timestep" in i:
      a.append(i)

In [ ]:
gauge_vis_path = save_folder+"*/*.pvd"
gauge_vis_path_collection = []

for folder_name in glob.iglob(gauge_vis_path, recursive=True):
    if os.path.isfile(folder_name):
        gauge_vis_path_collection.append(folder_name)
        print(folder_name)


combined_data = read_pvd_file(gauge_vis_path_collection[0])[:4]
for i in range(0,len(gauge_vis_path_collection)):
  add_lists_skip_random(combined_data, read_pvd_file(gauge_vis_path_collection[i])[3:-2]) #remove the last two lines and first three lines

combined_data = combined_data + read_pvd_file(gauge_vis_path_collection[0])[-2:]

In [ ]:
with open(save_folder+"combined.pvd",'w') as f:
  f.writelines(combined_data)